In [ ]:
import os
import numpy as np
import pandas as pd
os.chdir('/content')
!mkdir /root/.kaggle
!echo '{"username":"daniel1998xx","key":"c99226b7cd73b12d4681a78a5b02d227"}' > /root/.kaggle/kaggle.json
!kaggle datasets download -d shuyangli94/interview-npr-media-dialog-transcripts
if not os.path.exists("/content/NPR_Data"):
    os.makedirs("/content/NPR_Data")

mkdir: cannot create directory ‘/root/.kaggle’: File exists
interview-npr-media-dialog-transcripts.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip interview-npr-media-dialog-transcripts.zip -d NPR_Data

Archive:  interview-npr-media-dialog-transcripts.zip
replace NPR_Data/episodes.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
os.chdir('/content/NPR_Data')

In [ ]:
df_raw = pd.read_csv('utterances.csv')
df_raw.head()

,episode,episode_order,speaker,utterance
0,57264,9,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling M...","It's a 2,200-mile race. To give some sense of ..."
1,57264,10,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling M...",So for a top competitor like Lance to try to m...
2,57264,11,"NEAL CONAN, host","So in every team, presumably there's one star,..."
3,57264,12,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling M...",That's right. Each team has nine riders. And w...
4,57264,13,"NEAL CONAN, host","So slipstream, this is like drafting in car ra..."


In [ ]:
#df_raw.describe(include='all').round(2)

In [ ]:
df_raw.shape

(3199858, 4)

In [ ]:
df_raw.groupby('episode').count().sort_values(by='utterance', ascending=False) \
  .drop(axis=1, columns=['episode_order','speaker']).describe().round(2)

,utterance
count,104920.00
mean,30.49
std,32.82
min,1.00
25%,15.00
50%,21.00
75%,31.00
max,569.00


In [ ]:
df_raw.groupby('speaker').count().sort_values(by='utterance', ascending=False).drop(axis=1, columns=['episode_order','utterance'])

,episode
speaker,
"NEAL CONAN, host",206098
_NO_SPEAKER,184423
"IRA FLATOW, host",58292
"NEAL CONAN, HOST",43533
"FARAI CHIDEYA, host",36653
...,...
MARELLEN MAYERS,1
MAREK HALTER,1
MAREETA CARTER,1


In [ ]:
df_raw['speaker'] = df_raw['speaker'].str.upper()

In [ ]:
df_raw.groupby('speaker').count().sort_values(by='utterance', ascending=False).drop(axis=1, columns=['episode_order','utterance'])

,episode
speaker,
"NEAL CONAN, HOST",256525
_NO_SPEAKER,184423
"IRA FLATOW, HOST",73394
"STEVE INSKEEP, HOST",59492
"ROBERT SIEGEL, HOST",49687
...,...
"MS. KAYLENE JOHNSON (AUTHOR, ""SARAH"")",1
MS. KAYLON EPPINGER,1
MS. KAYTEE BUTE (CALLER),1


In [ ]:
df_raw['speaker'] = df_raw['speaker'].str.split(', H', expand=True)[0]

In [ ]:
df_raw.loc[df_raw['speaker'] == '_NO_SPEAKER']

,episode,episode_order,speaker,utterance
94,75004,7,_NO_SPEAKER,(Soundbite of hurricane forecaster Jose Rubier...
118,63416,25,_NO_SPEAKER,(Soundbite of music)
144,67560,22,_NO_SPEAKER,(SOUNDBITE OF ARCHIVED RECORDING)
151,67560,29,_NO_SPEAKER,(SOUNDBITE OF ARCHIVED RECORDING)
155,67560,33,_NO_SPEAKER,(SOUNDBITE OF ARCHIVED RECORDING)
...,...,...,...,...
3199834,64113,342,_NO_SPEAKER,(Soundbite of laughter)
3199836,64113,344,_NO_SPEAKER,(Soundbite of laughter)
3199838,64113,346,_NO_SPEAKER,(Soundbite of laughter)
3199848,67560,6,_NO_SPEAKER,(SOUNDBITE OF ARCHIVED RECORDING)


In [ ]:
df_raw = df_raw[df_raw['speaker'] != '_NO_SPEAKER']
df_raw.loc[df_raw['speaker'] == '_NO_SPEAKER']

,episode,episode_order,speaker,utterance


In [ ]:
df_raw['utterance_len'] = df_raw['utterance'].str.len()
df_raw.describe().round(2)

,episode,episode_order,utterance_len
count,3015435.00,3015435.00,3014882.00
mean,75665.61,33.44,201.78
std,37358.72,41.69,179.72
min,1.00,0.00,1.00
25%,56228.00,9.00,56.00
50%,75702.00,18.00,157.00
75%,105104.00,40.00,300.00
max,141179.00,569.00,8884.00


In [ ]:
df_raw.isnull().sum()

episode            0
episode_order      0
speaker            0
utterance        553
utterance_len    553
dtype: int64

In [ ]:
df_raw = df_raw.dropna()
df_raw.isnull().sum()

episode          0
episode_order    0
speaker          0
utterance        0
utterance_len    0
dtype: int64

In [ ]:
host_id_df = pd.read_json('host_id.json', orient='index')
host_id_df

,0
lulu garcia-navarro,0
melissa block,1
leila fadel,2
susan davis,3
sacha pfeiffer,4
...,...
clip from news broadcast,282
kathryn koob,283
dwight landry,284
kelly mccann,285


In [ ]:
host_id_df.reset_index(inplace=True)
host_id_df = host_id_df.rename(columns = {'index':'speaker'})
host_id_df

,speaker,0
0,lulu garcia-navarro,0
1,melissa block,1
2,leila fadel,2
3,susan davis,3
4,sacha pfeiffer,4
...,...,...
282,clip from news broadcast,282
283,kathryn koob,283
284,dwight landry,284
285,kelly mccann,285


In [ ]:
host_id_df['speaker'] = host_id_df['speaker'].astype('string')
host_id_df['speaker'] = host_id_df['speaker'].str.upper()
host_id_df.rename(columns={0:'host_id'}, inplace=True)
host_id_df

,speaker,host_id
0,LULU GARCIA-NAVARRO,0
1,MELISSA BLOCK,1
2,LEILA FADEL,2
3,SUSAN DAVIS,3
4,SACHA PFEIFFER,4
...,...,...
282,CLIP FROM NEWS BROADCAST,282
283,KATHRYN KOOB,283
284,DWIGHT LANDRY,284
285,KELLY MCCANN,285


In [ ]:
mergedf = df_raw.merge(host_id_df, how='left', on='speaker')
mergedf

,episode,episode_order,speaker,utterance,utterance_len,host_id
0,57264,9,"MS. LOREN MOONEY (EDITOR-IN-CHIEF, BICYCLING M...","It's a 2,200-mile race. To give some sense of ...",302.0,NaN
1,57264,10,"MS. LOREN MOONEY (EDITOR-IN-CHIEF, BICYCLING M...",So for a top competitor like Lance to try to m...,470.0,NaN
2,57264,11,NEAL CONAN,"So in every team, presumably there's one star,...",178.0,7.0
3,57264,12,"MS. LOREN MOONEY (EDITOR-IN-CHIEF, BICYCLING M...",That's right. Each team has nine riders. And w...,646.0,NaN
4,57264,13,NEAL CONAN,"So slipstream, this is like drafting in car ra...",58.0,7.0
...,...,...,...,...,...,...
3014877,67560,10,MARY LOUISE KELLY,On February 1 in Los Angeles...,31.0,10.0
3014878,67560,12,MARY LOUISE KELLY,UNIDENTIFIED MAN #2: Terrifying moments inside...,131.0,10.0
3014879,67560,13,MARY LOUISE KELLY,"That girl brought a gun to school, and it acci...",118.0,10.0
3014880,67560,14,ARI SHAPIRO,On January 31 - gunshots outside a high school...,87.0,9.0


In [ ]:
mergedf['host_id'] = mergedf['host_id'].fillna(-1)
mergedf

,episode,episode_order,speaker,utterance,utterance_len,host_id
0,57264,9,"MS. LOREN MOONEY (EDITOR-IN-CHIEF, BICYCLING M...","It's a 2,200-mile race. To give some sense of ...",302.0,-1.0
1,57264,10,"MS. LOREN MOONEY (EDITOR-IN-CHIEF, BICYCLING M...",So for a top competitor like Lance to try to m...,470.0,-1.0
2,57264,11,NEAL CONAN,"So in every team, presumably there's one star,...",178.0,7.0
3,57264,12,"MS. LOREN MOONEY (EDITOR-IN-CHIEF, BICYCLING M...",That's right. Each team has nine riders. And w...,646.0,-1.0
4,57264,13,NEAL CONAN,"So slipstream, this is like drafting in car ra...",58.0,7.0
...,...,...,...,...,...,...
3014877,67560,10,MARY LOUISE KELLY,On February 1 in Los Angeles...,31.0,10.0
3014878,67560,12,MARY LOUISE KELLY,UNIDENTIFIED MAN #2: Terrifying moments inside...,131.0,10.0
3014879,67560,13,MARY LOUISE KELLY,"That girl brought a gun to school, and it acci...",118.0,10.0
3014880,67560,14,ARI SHAPIRO,On January 31 - gunshots outside a high school...,87.0,9.0


In [ ]:
test = mergedf[mergedf['host_id'] != -1]
test.count()

episode          1174823
episode_order    1174823
speaker          1174823
utterance        1174823
utterance_len    1174823
host_id          1174823
dtype: int64

# **1. Reduce Dataset: In order for faster preprocessing and modeling, we want to reduce the size of the dataset**

In [ ]:
#only keep utterance with more than 100 utterence_len
test = test.loc[test['utterance_len'] >= 100]
test

,episode,episode_order,speaker,utterance,utterance_len,host_id
2,57264,11,NEAL CONAN,"So in every team, presumably there's one star,...",178.0,7.0
6,57264,15,NEAL CONAN,And so the guy who's in back has an easier tim...,276.0,7.0
8,57264,17,NEAL CONAN,"We're talking with Loren Mooney, the editor-in...",301.0,7.0
12,57264,21,NEAL CONAN,The ultimate expression - the training that yo...,102.0,7.0
19,57264,28,NEAL CONAN,"Bye-bye. There is also, as we mentioned, the t...",330.0,7.0
...,...,...,...,...,...,...
3014873,67560,5,MARY LOUISE KELLY,"On February 5, two shootings - one, a student ...",111.0,10.0
3014874,67560,7,MARY LOUISE KELLY,"UNIDENTIFIED MAN #1: That victim, said to be a...",119.0,10.0
3014876,67560,9,ARI SHAPIRO,That same day at a special education center in...,138.0,9.0
3014878,67560,12,MARY LOUISE KELLY,UNIDENTIFIED MAN #2: Terrifying moments inside...,131.0,10.0


In [ ]:
#only keep speaker with more than 10,000 observations (utterance)
top_speakers = test.groupby(['speaker']).size().loc[test.groupby(['speaker']).size() > 10000]
print (top_speakers.sort_values(ascending=False))
df = pd.DataFrame(test.loc[test['speaker'].isin(top_speakers.index.values)])
df = df.reset_index(drop=True)

speaker
NEAL CONAN         127313
STEVE INSKEEP       33941
ROBERT SIEGEL       30085
IRA FLATOW          30052
FARAI CHIDEYA       24019
MELISSA BLOCK       23242
RENEE MONTAGNE      21764
DAVID GREENE        19704
SCOTT SIMON         18628
RACHEL MARTIN       17388
AUDIE CORNISH       16716
GUY RAZ             16596
MICHEL MARTIN       16004
ARI SHAPIRO         13878
MADELEINE BRAND     13724
MICHELE NORRIS      12772
ALEX CHADWICK       12444
TONY COX            10594
ED GORDON           10412
dtype: int64


# **2. Cleaning the utterance**

In [ ]:
#use lowercase, almost always for nlp projects
df['utterance'] = df['utterance'].str.lower()

In [ ]:
#drop any utterance that starts with "unidentified", because it is not the host speaking. let know if you know a better way.
df['unidentified'] = df['utterance'].str.startswith('unidentified')
df = df[df['unidentified'] == False]
df

,episode,episode_order,speaker,utterance,utterance_len,host_id,unidentified
0,57264,11,NEAL CONAN,"so in every team, presumably there's one star,...",178.0,7.0,False
1,57264,15,NEAL CONAN,and so the guy who's in back has an easier tim...,276.0,7.0,False
2,57264,17,NEAL CONAN,"we're talking with loren mooney, the editor-in...",301.0,7.0,False
3,57264,21,NEAL CONAN,the ultimate expression - the training that yo...,102.0,7.0,False
4,57264,28,NEAL CONAN,"bye-bye. there is also, as we mentioned, the t...",330.0,7.0,False
...,...,...,...,...,...,...,...
469271,64113,349,NEAL CONAN,russell peters is currently touring north amer...,145.0,7.0,False
469272,64113,350,NEAL CONAN,"coming up, from ""evil dead"" to ""my name is bru...",208.0,7.0,False
469273,67560,1,ARI SHAPIRO,now we're going to update a list that we first...,124.0,9.0,False
469274,67560,4,ARI SHAPIRO,let's work backwards. on february 8 in the bro...,108.0,9.0,False


In [ ]:
final_df = df[['speaker', 'utterance']].copy()
final_df

,speaker,utterance
0,NEAL CONAN,"so in every team, presumably there's one star,..."
1,NEAL CONAN,and so the guy who's in back has an easier tim...
2,NEAL CONAN,"we're talking with loren mooney, the editor-in..."
3,NEAL CONAN,the ultimate expression - the training that yo...
4,NEAL CONAN,"bye-bye. there is also, as we mentioned, the t..."
...,...,...
469271,NEAL CONAN,russell peters is currently touring north amer...
469272,NEAL CONAN,"coming up, from ""evil dead"" to ""my name is bru..."
469273,ARI SHAPIRO,now we're going to update a list that we first...
469274,ARI SHAPIRO,let's work backwards. on february 8 in the bro...


# **3. Futher Cleaning and Preprocessing**

In [ ]:
#pip install contractions
#pip install autocorrect

In [ ]:
import re
import contractions
from autocorrect import Speller
# import nltk
# from nltk.tokenize import word_tokenize
# from nltk.stem import WordNetLemmatizer

ModuleNotFoundError: ignored

we need to remove ":", and others too

In [ ]:
#this should removes those words with parentheses like (laughing) or (ph), but its not doing it. anyone has an idea to fix this?
def remove_text_within_parentheses(text):
    Formatted_text = re.sub(r"\([^()]*\)", "", text)
    return Formatted_text

In [ ]:
def remove_whitespace(text):
    pattern = re.compile(r'\s+') 
    Without_whitespace = re.sub(pattern, ' ', text)
    return Without_whitespace

In [ ]:
# def reducing_incorrect_character_repeatation(text):
#     Pattern_alpha = re.compile(r"([A-Za-z])\1{1,}", re.DOTALL)
#     Formatted_text = Pattern_alpha.sub(r"\1\1", text) 
#     Pattern_Punct = re.compile(r'([.,/#!$%^&*?;:{}=_`~()+-])\1{1,}')
#     Combined_Formatted = Pattern_Punct.sub(r'\1', Formatted_text)
#     Final_Formatted = re.sub(' {2,}',' ', Combined_Formatted)
#     return Final_Formatted

In [ ]:
def expand_contraction(text):
    expanded_words = []   
    for word in text.split():
      expanded_words.append(contractions.fix(word))  
    expanded_text = ' '.join(expanded_words)
    return expanded_text

In [ ]:
def spelling_correction(text):
    spell = Speller(lang='en')
    Corrected_text = spell(text)
    return Corrected_text

In [ ]:
# w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
# lemmatizer = nltk.stem.WordNetLemmatizer()
# def lemmatization(text):
#     lemma = [lemmatizer.lemmatize(w,'v') for w in w_tokenizer.tokenize(text)]
#     return lemma

In [ ]:
def text_preprocessing(text, contractions=True, extra_whitespace=True, parantheses=True, mis_spell=True):
    if parantheses == True:
        Data = remove_text_within_parentheses(text)
        
    if extra_whitespace == True:
        Data = remove_whitespace(Data)

    if contractions == True:
        Data = expand_contraction(Data)
        
    spell = Speller(lang='en')
    
    if mis_spell == True:
        Data = spelling_correction(Data)
    return Data

In [ ]:
utterance_list = final_df['utterance'].to_list()
final_list = []
complete_list = []
for utterance in utterance_list:
    processed = text_preprocessing(utterance)
    final_list.append(processed)
complete_list.extend(final_list)
final_df['processed_utterance'] = complete_list

NameError: ignored

In [ ]:
final_df

In [ ]:
# for item in final_df['utterance']:
#   print(item)

# **Modeling: finding the best parameter**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
#from sklearn.cross_validation import cross_val_score
from sklearn.pipeline import make_pipeline

In [ ]:
X = final_df.utterance
y = final_df.speaker
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
pipe = make_pipeline(TfidfVectorizer(), MultinomialNB())
pipe.steps

param_grid = {}
param_grid["tfidfvectorizer__max_features"] = [500, 1000, 15000]
param_grid["tfidfvectorizer__ngram_range"] = [(1,1), (1,2), (2,2)]
param_grid["tfidfvectorizer__stop_words"] = ["english", None]
param_grid["tfidfvectorizer__strip_accents"] = ["ascii", "unicode", None]
param_grid["tfidfvectorizer__analyzer"] = ["word", "char"]
param_grid["tfidfvectorizer__binary"] = [True, False]
param_grid["tfidfvectorizer__norm"] = ["l1", "l2", None]
param_grid["tfidfvectorizer__use_idf"] = [True, False]
param_grid["tfidfvectorizer__smooth_idf"] = [True, False]
param_grid["tfidfvectorizer__sublinear_tf"] = [True, False]

grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy')
grid.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__memory', 'estimator__steps', 'estimator__verbose', 'estimator__tfidfvectorizer', 'estimator__multinomialnb', 'estimator__tfidfvectorizer__analyzer', 'estimator__tfidfvectorizer__binary', 'estimator__tfidfvectorizer__decode_error', 'estimator__tfidfvectorizer__dtype', 'estimator__tfidfvectorizer__encoding', 'estimator__tfidfvectorizer__input', 'estimator__tfidfvectorizer__lowercase', 'estimator__tfidfvectorizer__max_df', 'estimator__tfidfvectorizer__max_features', 'estimator__tfidfvectorizer__min_df', 'estimator__tfidfvectorizer__ngram_range', 'estimator__tfidfvectorizer__norm', 'estimator__tfidfvectorizer__preprocessor', 'estimator__tfidfvectorizer__smooth_idf', 'estimator__tfidfvectorizer__stop_words', 'estimator__tfidfvectorizer__strip_accents', 'estimator__tfidfvectorizer__sublinear_tf', 'estimator__tfidfvectorizer__token_pattern', 'estimator__tfidfvectorizer__tokenizer', 'estimator__tfidfvectorizer__use_idf', 'estimator__tfidfvectorizer__

In [ ]:
#find the best parameters
grid.fit(X,y)

KeyboardInterrupt: ignored

In [ ]:
print(grid.best_params_)
print(grid.best_score_)

In [ ]:
#use the best parameters and define model
vector = TfidfVectorizer(analyzer='word', stop_words='english', max_features = 850, ngram_range=(1, 1), 
                       binary=False, norm=None, smooth_idf=True, strip_accents=None,
                       sublinear_tf=True, use_idf=False)

df_transformed = vector.fit_transform(X)

In [ ]:
multi_nb_model = MultinomialNB()
multi_nb_model.fit(df_transformed, y)
print ("Model accuracy within dataset: ", multi_nb_model.score(df_transformed, y))

# **TEST MODEL**

In [ ]:
test_text = ["blah blah blahh"]
test_text_transform = vector.transform(test_text)

print (multi_nb_model.predict(test_text_transform)," most likely said it.")